# My Spotify Data
A project to analyze my own spotify data and see what I can learn from it.

List of things I want to do:
1. create dataframe
2. do monthly playlists show pattterns?
3. what charactersistics have my favourite songs/artists/...

4. update function
    - update dataframe with new playlists from my library
    - only add new data


overall: 
- create exploratory analysis and get recommended playlist for a month based on my listening habits
- create dashboard/overview/... that displays all the interesting stuff in a visually appealing manner


In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

import math
import pandas as pd
import numpy as np


## 1 Spotify Data
using ```spotipy``` library to get my spotify data.

ID: c9a30106e12d4683aeec87d37a2749e6 <br>
secret: ea7e2ddc2fa14d5ebb0172a079763088

## 1.1 authentication

In [2]:
# scope = "user-library-read"	
scope = "playlist-read-private"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="c9a30106e12d4683aeec87d37a2749e6",
                                               client_secret="ea7e2ddc2fa14d5ebb0172a079763088",
                                               redirect_uri="http://localhost:8080/",
                                               scope=scope))



## 1.2 get playlists

create dictionary, containing the name and id of each playlist

In [3]:
results = sp.current_user_playlists(limit=50)

my_playlists = {}

for i in range(math.ceil(results['total']/50)):
    playlist_results = sp.current_user_playlists(limit=50, offset=i*50)
    for playlist in playlist_results['items']:
        my_playlists[playlist['name']] = playlist['id']


In [4]:
166/50


3.32

In [5]:
results

{'href': 'https://api.spotify.com/v1/users/*lotti_karotti*/playlists?offset=0&limit=50',
 'limit': 50,
 'next': 'https://api.spotify.com/v1/users/*lotti_karotti*/playlists?offset=50&limit=50',
 'offset': 0,
 'previous': None,
 'total': 166,
 'items': [{'collaborative': False,
   'description': '',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/0C5N4S6WzVD5UJhvrGLKdF'},
   'href': 'https://api.spotify.com/v1/playlists/0C5N4S6WzVD5UJhvrGLKdF',
   'id': '0C5N4S6WzVD5UJhvrGLKdF',
   'images': [{'height': 640,
     'url': 'https://mosaic.scdn.co/640/ab67616d00001e02136f35f358df39f6cc93192dab67616d00001e02323ee807f1eba5e70e9884ecab67616d00001e024ea5a8e36f55476098bfe914ab67616d00001e028a0373922e28fe2e88d46aef',
     'width': 640},
    {'height': 300,
     'url': 'https://mosaic.scdn.co/300/ab67616d00001e02136f35f358df39f6cc93192dab67616d00001e02323ee807f1eba5e70e9884ecab67616d00001e024ea5a8e36f55476098bfe914ab67616d00001e028a0373922e28fe2e88d46aef',
     'width': 300},
   

In [6]:
len(my_playlists)

# for i, (k, v) in enumerate(my_playlists.items()):
#     print(k, v)
#     if i > 3:
#         break

164

## 1.3 get tracks for each playlist

data structure: <br>
tracks_df:      contains each track and all infos on track <br>
playlist_df:    contains all tracks of a playlist


In [7]:
playlist_df = pd.DataFrame(my_playlists.items(), columns=['name', 'id'])
playlist_df.to_csv('data/playlists_df.csv', index=False, header=True)


clean playlist_df:
- only include monthly playlists
or
- only include playlists created by me - chose that for now
- and remove type=episode (has no audio features) + add extra check for audio features -> on track level

In [13]:
for playlist_id in list(my_playlists.values()):
    playlist = sp.playlist(playlist_id)
    if playlist['owner']['id'] != '*lotti_karotti*':
        my_playlists.pop(playlist['name'])

len(my_playlists)

In [45]:
tracks = [] # list of track ids (unique) from all playlists
for playlist_name, playlist_id in my_playlists.items():
    playlist = sp.playlist_tracks(playlist_id)
    for track in playlist['items']:
        track_id = track['track']['id']
        if track_id not in tracks and track['track']['type'] == 'track':
            tracks.append(track_id)

## 1.4 get audio features for each track

In [52]:
# createtracks dataframe from list
tracks_df = pd.DataFrame(tracks, columns=['track_id'])
tracks_df.to_csv('data/tracks_df.csv', index=False, header=True)

# get audio features for all tracks
audio_features = []
for i in range(math.ceil(len(tracks)/100)):
    features = sp.audio_features(tracks[i*100:(i+1)*100])
    for feature in features:
        if feature is not None:
            audio_features.append(feature)

# add audio features to dataframe
tracks_df = pd.concat([tracks_df, pd.DataFrame(audio_features)], axis=1)
tracks_df.to_csv('data/tracks_df.csv', index=False, header=True)


## 1.5 get additional features for each track

In [ ]:
features = []

for uri in tracks_df['uri']:
    track = sp.track(uri)
    name = track['name']
    artist = track['artists'][0]['name']
    artist_id = track['artists'][0]['id']
    artist_info = sp.artist(track['artists'][0]['id'])
    popularity = track['popularity']
    album = track['album']['name']
    album_id = track['album']['id']


In [ ]:
for 
track = f'spotify:track:{id}'

## medium article code snippets


get info from track id

In [ ]:
id = 'spotify:track:5T0BeNEv0U7xuiEwlCFBCc'
track = sp.track(id)
print(track['name'])

### extracting tracks from a playlist

In [ ]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

extract the name of each track, the name of the album that it belongs to, and the popularity of the track

In [ ]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]

### extracting audio features from a track

In [ ]:
sp.audio_features(track_uri)[0]